In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np

In [44]:
from labcore.measurement import Sweep, pointer, record_as, indep, sweep_parameter, recording, dep
from labcore.measurement.sweep import AsyncRecord

## configuring sweeps

In [15]:
@recording('x')
def measure_random_numbers(mean=0, scale=1., nvals=1):
    return np.random.normal(loc=mean, scale=scale, size=nvals)

In [22]:
sweep = sweep_parameter('a', range(3), measure_random_numbers)
for s in sweep:
    print(s)

{'a': 0, 'x': array([-1.07520604])}
{'a': 1, 'x': array([-0.98872233])}
{'a': 2, 'x': array([1.78346645])}


In [23]:
sweep = sweep_parameter('a', range(3), measure_random_numbers.using(mean=10))
for s in sweep:
    print(s)

{'a': 0, 'x': array([9.50424447])}
{'a': 1, 'x': array([9.88050777])}
{'a': 2, 'x': array([10.46168569])}


In [24]:
sweep = sweep_parameter('a', range(3), measure_random_numbers)
sweep.set_options(
    measure_random_numbers=dict(mean=-5),
)
for s in sweep:
    print(s)

{'a': 0, 'x': array([-3.81455827])}
{'a': 1, 'x': array([-4.34206255])}
{'a': 2, 'x': array([-5.90485879])}


In [25]:
sweep = sweep_parameter('a', range(3), measure_random_numbers.using(mean=10))
sweep.set_options(
    measure_random_numbers=dict(mean=-5),
)
for s in sweep:
    print(s)

{'a': 0, 'x': array([11.06953062])}
{'a': 1, 'x': array([8.74032162])}
{'a': 2, 'x': array([8.66727255])}


## Lazy pointer

In [33]:
def a_generator_function(n_vals=3):
    x = 0
    while x < n_vals:
        yield x
        x += 1

sweep = sweep_parameter('x', a_generator_function())

print('doing the first sweep:')
for s in sweep:
    print(s)

print('\ntrying again:')
for s in sweep:
    print(s)

doing the first sweep:
{'x': 0}
{'x': 1}
{'x': 2}

trying again:


In [76]:
@pointer('x')
def a_generator_function(n_vals=3):
    x = 0
    while x < n_vals:
        yield x
        x += 1

sweep = Sweep(a_generator_function)

print('doing the first sweep:')
for s in sweep:
    print(s)

print('\ntrying again:')
for s in sweep:
    print(s)

doing the first sweep:
{'x': 0}
{'x': 1}
{'x': 2}

trying again:
{'x': 0}
{'x': 1}
{'x': 2}


In [77]:
sweep = Sweep(a_generator_function.using(n_vals=5))

print('doing the first sweep:')
for s in sweep:
    print(s)

doing the first sweep:
{'x': 0}
{'x': 1}
{'x': 2}
{'x': 3}
{'x': 4}


## Complicated pointer 

## AsyncRecord

In [35]:
import time

In [132]:
class DummyInstrument:
    
    def __init__(self, intervals):
        self.intervals = intervals
        
    def run(self):
        t0 = time.time()
        for i in self.intervals:
            while time.time()-t0 < i:
                time.sleep(0.01)
            yield time.time()


class DelayedGatheringOfTimes(AsyncRecord):
    
    def setup(self, fun, *args, **kwargs):
        self.communicator['time'] = time.time()
        intervals = fun(*args, **kwargs)
        self.communicator['instrument'] = DummyInstrument(intervals)
    
    def collect(self, print_data=False, **kwargs):
        for data in self.communicator['instrument'].run():
            if print_data:
                print('data:', data)
            yield data
        

@DelayedGatheringOfTimes(
    indep('interrogation_times'),
)
def equally_spaced_times(wait_time=0.5, points=2):
    return np.arange(points) * wait_time


sweep = equally_spaced_times(wait_time=0.2, points=10, collector_options={'print_data': False})

sweep.set_options(
    equally_spaced_times=dict(points=5)
)

for s in sweep:
    print(s)

{'interrogation_times': None}
{'interrogation_times': 1630428893.361867}
{'interrogation_times': 1630428893.562287}
{'interrogation_times': 1630428893.7699819}
{'interrogation_times': 1630428893.9668071}
{'interrogation_times': 1630428894.16322}
{'interrogation_times': 1630428894.368414}
{'interrogation_times': 1630428894.572625}
{'interrogation_times': 1630428894.7647948}
{'interrogation_times': 1630428894.965135}
{'interrogation_times': 1630428895.1685271}


In [137]:
outer_sweep = sweep_parameter('points', range(1,5))
inner_sweep = equally_spaced_times(wait_time=0.2)
combined_sweep = outer_sweep @ inner_sweep

for data in combined_sweep:
    print(data)

{'points': 1, 'interrogation_times': None}
{'points': 1, 'interrogation_times': 1630429139.222152}
{'points': 2, 'interrogation_times': None}
{'points': 2, 'interrogation_times': 1630429139.2222779}
{'points': 2, 'interrogation_times': 1630429139.426282}
{'points': 3, 'interrogation_times': None}
{'points': 3, 'interrogation_times': 1630429139.427327}
{'points': 3, 'interrogation_times': 1630429139.631752}
{'points': 3, 'interrogation_times': 1630429139.8366048}
{'points': 4, 'interrogation_times': None}
{'points': 4, 'interrogation_times': 1630429139.837495}
{'points': 4, 'interrogation_times': 1630429140.047543}
{'points': 4, 'interrogation_times': 1630429140.24167}
{'points': 4, 'interrogation_times': 1630429140.441689}
